#### KPI 4: CO2 emissions (top down)
---
Combined data frame for every entity in use case with:
- grey_energy [kWh] -> to be multiplied by co2 emission factor
- grey_energy [%] -> share grey energy consumed (1 - share renewable)

In [179]:
import os
import pandas as pd

from functions_topdown_v2 import *

home_path_new = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\new_results_2023'
results_new = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\new_results_2023\results'

home_path_old = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen'
results_old = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\results'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Instructions how to use function ```co2_calcuation```:
- enter the relevant simulation data directory for old results (```home_path_old```) or new results (```home_path_new```)
- enter the relevant use case number as an integer
- save the result in a pandas dataframe
- save the dataframe as a xlsx-file to old results folder (path: ```results_old```) or new results folder (path: ```results_new```)

In [180]:
# calculate co2 emissions for use case
df4 = co2_calculation(home_path_new, use_case_nr=4)

In [ ]:
df4

In [ ]:
# save dataframe to xlsx-file in results path
df4.to_excel(os.path.join(results_new, 'use_case_4_topdown.xlsx'))

---

In [173]:
months=4
use_case_dir, use_case_nr = get_use_case_dir_and_nr(home_path_new, use_case_nr=4)
df_out = prepare_df_out(use_case_dir)
nonempty_files_paths, nr_nonempty_files, empty_files_paths = prepare_empty_and_nonempty_filepaths(use_case_dir, use_case_nr)
mm_filepaths = [i for i in nonempty_files_paths if 'member' in i.split('\\')[-1]]
nr_mm_filepaths = len(mm_filepaths)

# set up df_mm
df_mm = df_out.copy()
df_mm[['mm_to_cs', 'cs_to_mm']] = 0, 0

# for each entity involved with mm, combine all months' data frames
entity_dfs = []
for i in range(int(nr_mm_filepaths / months)):
    entity_monthly_dfs = []
    entity_monthly_filepaths = mm_filepaths[i::int(nr_mm_filepaths / months)]
    for j in entity_monthly_filepaths:
        df_temp = pd.read_csv(j).drop(['creation_time', 'matching_requirements', 'rate [ct./kWh]'], axis=1)
        df_temp.seller = [i.lower().replace('_', '-') for i in df_temp.seller]
        df_temp.buyer = [i.lower().replace('_', '-') for i in df_temp.buyer]
        entity_monthly_dfs.append(df_temp)
    entity_name = entity_monthly_filepaths[0].split('\\')[-1].split('-trades.csv')[0]
    entity_df = pd.concat(entity_monthly_dfs).sort_values(by='slot').reset_index(drop=True)
    entity_dfs.append(entity_df)

# combine all dfs and calculate mm trading volumes
mm_df = pd.concat(entity_dfs, axis=0).sort_values(by='slot')
mm_df.set_index(['slot'], inplace=True)

In [154]:
# obtain all months' time slots from .json files and combine them to index
json_filepaths, timeslots, timeslots_len = [], [], []
for root, dir, file in os.walk(top=use_case_dir, topdown=True):
    json_filepaths += [os.path.join(root, f) for f in file if 'price_energy_day.json' in f]
for j in json_filepaths:
    current_month = j.split('\\aggregated')[0].split('\\')[-1]
    
    df = pd.read_json(j)
    timeslots_j = [df['Germany']['price-energy-day'][i][0]['time'] for i in
                    range(len(df['Germany']['price-energy-day']))]
    timeslots.extend(timeslots_j) 
    timeslots_len.append(len(timeslots_j))

In [170]:
j = json_filepaths[0]
current_month_str = j.split('\\aggregated')[0].split('\\')[-1].lower()
print(current_month_str)
current_month_number = None
if 'jan' in current_month_str:
    current_month_number = '01'
elif 'apr' in current_month_str:
    current_month_number = '04'
elif 'jul' in current_month_str:
    current_month_number = '07'
elif 'oct' in current_month_str:
    current_month_number = '10'
else:
    raise Exception(f'ERROR: Month could not be detected from string {current_month_str}.')
df = pd.read_json(j)
timeslots_j = [df['Germany']['price-energy-day'][i][0]['time'] for i in
                    range(len(df['Germany']['price-energy-day']))]
# replace month number in string with detected month number from filepath
[i.split('-')[0] + '-' + current_month_number + '-' + i.split('-')[-1] for i in timeslots_j]


apr


Find missing files

In [ ]:
use_case_dir, use_case_nr = get_use_case_dir_and_nr(home_path_new, use_case_nr=6)

all_filepaths, all_filenames = [], []
for root, dir, file in os.walk(top=use_case_dir, topdown=True):
    all_filepaths += [os.path.join(root, f) for f in file if 'trades.csv' in f]
    all_filenames += [f for f in file if 'trades.csv' in f]
    
nr_months = 4
months = {'apr', 'jul', 'oct', 'jan'}
incomplete_fnames = [i for i in all_filenames if all_filenames.count(i) != nr_months]

for f in incomplete_fnames:
    present_months = set([i.split('\\germany')[0][-3:] for i in all_filepaths if f in i])
    missing_months = months - present_months
    print(f'{f}:\n',f'- present_months: {list(present_months)}\n',f'- missing_months: {list(missing_months)}\n')